In [1]:
import numpy
import math
import pandas

In [2]:
ucca_train = pandas.read_csv(r'C:\Users\jyellin\re_1\Code\path_to_re\train-ucca_paths_per_relation.csv')
ucca_test = pandas.read_csv(r'C:\Users\jyellin\re_1\Code\path_to_re\test-ucca_paths_per_relation.csv')
jaccard_weighted_scores = []

In [3]:
ucca_train['source'] = 'train'

In [4]:
ucca_test['source'] = 'test'

In [5]:
ucca = pandas.concat([ucca_train, ucca_test])

In [6]:
ucca_stats_by_relation_and_path = ucca.groupby(['source','relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','source', 'total'], ascending=[True,True,False]).set_index(['relation','source'])
ucca_stats_by_relation = ucca.groupby(['source','relation'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','source'], ascending=[True, True]).set_index(['relation','source'])

In [7]:
paths = ucca_stats_by_relation_and_path['path'].unique().tolist()
relations = ucca_stats_by_relation.index.get_level_values(0).unique().tolist()

path_to_index = {k: v for v, k in enumerate(paths)}
relation_to_index = {k: v for v, k in enumerate(relations)}

totals = ucca_stats_by_relation['total'].to_list()
mean_total = numpy.mean(totals)

In [8]:
relation_to_count_matrix_train = numpy.zeros((len(relations), len(paths)), dtype=numpy.intc)

for index, row in ucca_stats_by_relation_and_path.iterrows():
    
    if index[1] == 'test':
        continue
    
    relation = index[0]
    path = row['path']
    
    relation_index = relation_to_index[relation]
    path_index = path_to_index[path]
    
    total = row['total']
    if total == 1:
        total = 0
    relation_to_count_matrix_train[relation_index, path_index] = total

relation_to_count_df_train = pandas.DataFrame(relation_to_count_matrix_train, index=[relations,['train']*len(relations)], columns=paths)


In [9]:
relation_to_count_matrix_test = numpy.zeros((len(relations), len(paths)), dtype=numpy.intc)

for index, row in ucca_stats_by_relation_and_path.iterrows():
    
    if index[1] == 'train':
        continue
    
    relation = index[0]
    path = row['path']
    
    relation_index = relation_to_index[relation]
    path_index = path_to_index[path]
    
    total = row['total']
    if total == 1:
        total = 0
    relation_to_count_matrix_test[relation_index, path_index] = total

relation_to_count_df_test = pandas.DataFrame(relation_to_count_matrix_test, index=[relations,['test']*len(relations)], columns=paths)


In [10]:
relation_to_count_df = pandas.concat([relation_to_count_df_train, relation_to_count_df_test])

In [11]:
# create a single data frame from the total-relation counts and the relation-to-cound dataframe
temp = pandas.concat([ucca_stats_by_relation,relation_to_count_df], axis=1)

# filter out all relations for which there are less than 25% of the mean total sentences in the training set
temp = temp[temp['total'] >= mean_total/4]

# normalize counts 
temp = temp.apply( lambda x: x / x['total'] * mean_total, axis=1).drop(columns=['total'])

# round up to 
relation_to_count_normalized = temp.round(0).astype(int)

In [12]:
paths = ucca_stats_by_relation_and_path['path'].unique().tolist()
relations = ucca_stats_by_relation.index.get_level_values(0).unique().tolist()

for relation in relations:
    
    test_row = relation_to_count_normalized[
        (relation_to_count_normalized.index.get_level_values(0) == relation) &
        (relation_to_count_normalized.index.get_level_values(1) == 'test')]
    
    train_row = relation_to_count_normalized[
        (relation_to_count_normalized.index.get_level_values(0) == relation) &
        (relation_to_count_normalized.index.get_level_values(1) == 'train')]

    
    if (len(test_row) == 0 or len(train_row) == 0):
        continue
    
    numerator = 0
    denominator = 0
    
    for path in paths:
        test_row_count = 0
        if len(test_row[path].values) > 0:
            test_row_count = test_row[path].values[0]
            
        train_row_count = 0
        if len(train_row[path].values) > 0:
            train_row_count = train_row[path].values[0]
        
        numerator = numerator + min(test_row_count, train_row_count)
        denominator = denominator + max(test_row_count, train_row_count)
    
    if denominator > 0:
        jaccard_weighted_scores.append((relation, '{} (test)'.format(relation), numerator / denominator ))
        #scores[relation] = numerator / denominator 

In [13]:
#score_df = pandas.DataFrame.from_dict(scores,  orient='index', columns=['score']).sort_values(['score'], ascending=[False] )
#score_df

In [14]:
ucca_train_stats_by_relation_and_path = ucca_train.groupby(['relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).reset_index(drop=True)
ucca_train_stats_relation = ucca_train.groupby(['relation'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation'], ascending=[True]).set_index('relation')

In [15]:
paths = ucca_train_stats_by_relation_and_path['path'].unique().tolist()
relations = ucca_train_stats_by_relation_and_path['relation'].unique().tolist()

path_to_index = {k: v for v, k in enumerate(paths)}
relation_to_index = {k: v for v, k in enumerate(relations)}

totals = ucca_train_stats_relation['total'].to_list()
mean_total = numpy.mean(totals)

In [16]:
relation_to_count_matrix = numpy.zeros((len(relations), len(paths)), dtype=numpy.intc)

for index, row in ucca_train_stats_by_relation_and_path.iterrows():
    relation_index = relation_to_index[row['relation']]
    path_index = path_to_index[row['path']]
    
    total = row['total']
    if total <= 1:
        total = 0
    relation_to_count_matrix[relation_index, path_index] = total

relation_to_count_df = pandas.DataFrame(relation_to_count_matrix, index=relations, columns=paths)
#relation_to_count_matrix = numpy.concatenate((numpy.reshape(totals, (-1,1)),relation_to_count_matrix), axis=1)

In [17]:
# create a single data frame from the total-relation counts and the relation-to-cound dataframe
temp = pandas.concat([ucca_train_stats_relation,relation_to_count_df], axis=1)

# filter out all relations for which there are less than 25% of the mean total sentences in the training set
temp = temp[temp['total'] >= mean_total/4]

# normalize counts 
temp = temp.apply( lambda x: x / x['total'] * mean_total, axis=1).drop(columns=['total'])

# round up to 
relation_to_count_normalized = temp.round(0).astype(int)

In [18]:
relations = relation_to_count_normalized.index.to_list()
paths = relation_to_count_normalized.columns.to_list()
work = relation_to_count_normalized.to_numpy()

for i, i_relation in enumerate(relations):
    i_row = work[i]

    for j, j_relation in enumerate(relations[i+1:]):
        j_row = work[i+1+j]

        numerator = 0
        denominator = 0
        for k, k_path in enumerate(paths):
            numerator = numerator + min(i_row[k], j_row[k])
            denominator = denominator + max(i_row[k], j_row[k])
        
        jaccard_weighted_scores.append((i_relation, j_relation, numerator / denominator ))
        



In [19]:
from operator import itemgetter

jaccard_weighted_scores = sorted(jaccard_weighted_scores, key=itemgetter(2), reverse=True)
jaccard_weighted_scores_df = pandas.DataFrame.from_records(jaccard_weighted_scores, columns =['rel1', 'rel2', 'jaccard']) 

In [20]:
pandas.set_option('display.max_rows', 100)
jaccard_weighted_scores_df.head(100)

,rel1,rel2,jaccard
0,per:title,per:title (test),0.612565
1,org:alternate_names,org:alternate_names (test),0.590244
2,org:top_members/employees,org:top_members/employees (test),0.539604
3,per:employee_of,per:employee_of (test),0.507937
4,per:countries_of_residence,per:employee_of,0.477352
5,per:age,per:age (test),0.446512
6,per:origin,per:origin (test),0.438202
7,per:countries_of_residence,per:stateorprovinces_of_residence,0.432143
8,per:cities_of_residence,per:cities_of_residence (test),0.429379
9,per:countries_of_residence,per:countries_of_residence (test),0.420732
